In [1]:
import pandas as pd
import sys
import os
import time
import faiss
import heapq
from sklearn.metrics.pairwise import cosine_similarity
from data_preprocess import DataPreprocessing
from io import StringIO
pre_process = DataPreprocessing()
pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.width', None)
from IPython.display import display, HTML

In [2]:
master_data = pd.read_csv("input_data/master_data.csv")

master_data = master_data[['DIVISION_SHORT_DESCR', 'SUB_DIVISION_SHORT_DESCR', 'BUSINESS_SEGMENT',
             'HFGR', 'HFGR_SHORT_DESCR', 'FGR', 'FGR_DESC', 'MPI_TYPE',
             'PRODUCT_FAMILY', 'APPLICATION', 'TECHNOLOGY', 'PROCESSCLASS',
             'PROCESSGROUP', 'SALES_NAME', 'MSAI_SP_SAP_MATNR', 'MBNO_FP_SAP_MATNR',
             'DC_BNR_MAX', 'BASIC_TYPE_BE', 'MPK_PACKAGE_PLATFORM',
             'PACKAGE_FAMILY_PROCESSED', 'DC_V_SNR_SERIAL_NO', 'PACKAGENAME',
             'QA_MILESTONE', 'QUALITY_REQ_CATEGORY', 'M8_RELEASE_DATE',
             'M9_RELEASE_DATE', 'MSAI_FUNCTIONAL_SAFETY', 'MSAI_PRODUCT_STATUS_INFO',
             'BE_PVG_VALID_TO', 'BASIC_TYPE_DB', 'TOTAL_CHIPS_PROCESSED',
             'MULTICHIP_PROCESSED', 'BASIC_TYPE_BREAKDOWN_PROCESSED', 'FAB_LOC',
             'ASSEMBLY', 'TEST', 'MAIN_FAMILY', 'ENDTHICKNESS_PRAS_MIN',
             'PRWF_RASTER_X', 'PRWF_RASTER_Y', 'MBFS_FRONTEND_QTY', 'PRWF_CHIP_SIZE',
             'MANUFACTURING_ROUTE']]


test_input = master_data.head(1)

# Label Encoder / Scaling

### FAISS APPROACH

In [10]:
label_encoded_master_data = pd.read_csv('label_encoding/encoded_data.csv')
labelindex = faiss.IndexFlatL2(label_encoded_master_data.shape[1])
labelindex.add(label_encoded_master_data)
preprocessed, error = pre_process.label_encoder_vector_conversion(test_input, "inference")
input_data_values = preprocessed.values
start_time = time.time()
distances, indices = labelindex.search(input_data_values, 20)
end_time = time.time()
print(f"Execution Time: {end_time - start_time} seconds")
print(f'Recommended Index {indices.tolist()[0]}')
print(f'Distance List{distances.tolist()[0]}')
selected_rows = master_data.iloc[indices.tolist()[0]]
selected_rows

['DIVISION_SHORT_DESCR', 'SUB_DIVISION_SHORT_DESCR', 'HFGR', 'HFGR_SHORT_DESCR', 'FGR', 'FGR_DESC', 'MPI_TYPE', 'PRODUCT_FAMILY', 'APPLICATION', 'TECHNOLOGY', 'PROCESSCLASS', 'PROCESSGROUP', 'SALES_NAME', 'MSAI_SP_SAP_MATNR', 'MBNO_FP_SAP_MATNR', 'BASIC_TYPE_BE', 'MPK_PACKAGE_PLATFORM', 'PACKAGE_FAMILY_PROCESSED', 'DC_V_SNR_SERIAL_NO', 'PACKAGENAME', 'QA_MILESTONE', 'QUALITY_REQ_CATEGORY', 'M8_RELEASE_DATE', 'M9_RELEASE_DATE', 'MSAI_FUNCTIONAL_SAFETY', 'MSAI_PRODUCT_STATUS_INFO', 'BE_PVG_VALID_TO', 'BASIC_TYPE_DB', 'MULTICHIP_PROCESSED', 'BASIC_TYPE_BREAKDOWN_PROCESSED', 'FAB_LOC', 'ASSEMBLY', 'TEST', 'MAIN_FAMILY']
Execution Time: 0.0011649131774902344 seconds
Recommended Index [0, 34283, 23503, 39633, 11183, 39061, 43858, 18678, 25032, 27314, 39563, 8940, 6557, 30521, 18672, 31481, 43058, 4979, 34144, 24031]
Distance List[0.0, 1.658600091934204, 1.8302000761032104, 2.053499937057495, 2.493499994277954, 2.667800188064575, 2.8407998085021973, 3.20419979095459, 3.213700294494629, 3.3119

DIVISION_SHORT_DESCR SUB_DIVISION_SHORT_DESCR  BUSINESS_SEGMENT HFGR  \
0                      PSS                       PS                18    A   
34283                  PSS                       PS                18    A   
23503                  PSS                       PS                18    A   
39633                  PSS                       PS                18    A   
11183                  PSS                       PS                18    A   
39061                  PSS                       PS                18    A   
43858                  PSS                       PS                15    A   
18678                  PSS                       PS                18    A   
25032                  PSS                       PS                18    A   
27314                  PSS                       PS                18    A   
39563                  PSS                       PS                18    A   
8940                   PSS                       PS                18    A   
6557                   PSS                       PS                18    A   
30521                  PSS                       PS                18    A   
18672                  PSS                       PS                18    A   
31481                  PSS                       PS                18    A   
43058                  PSS                       PS                15    A   
4979                   PSS                       PS                18    A   
34144                  PSS                       PS                18    A   
24031                  PSS                       PS                15    A   

      HFGR_SHORT_DESCR  FGR   FGR_DESC MPI_TYPE PRODUCT_FAMILY  \
0            Trench LV  A8N  102083_16   Device            75V   
34283        Trench LV  AFV  102627_16   Device            75V   
23503        Trench LV  A8N  102083_16   Device            75V   
39633        Trench LV  A8N  102083_16   Device            75V   
11183        Trench LV  AD2  102418_16   Device            60V   
39061        Trench LV  A2C  101428_16   Device            55V   
43858       Trench ULV  A87  102067_16   Device            40V   
18678        Trench LV  A8N  102083_16   Device            75V   
25032        Trench LV  A6Q  102002_16   Device            60V   
27314        Trench LV  A2C  101428_16   Device            55V   
39563        Trench LV  A6Q  102002_16   Device            60V   
8940         Trench LV  A6M  101996_16   Device            60V   
6557         Trench LV  A1G  101350_16   Device            75V   
30521        Trench LV  AFV  102627_16   Device            75V   
18672        Trench LV  A6S  102004_16   Device            75V   
31481        Trench LV  A6Q  102002_16   Device            60V   
43058       Trench ULV  AEB  102559_16   Device            40V   
4979         Trench LV  A8N  102083_16   Device            75V   
34144        Trench LV  A6S  102004_16   Device            75V   
24031       Trench ULV  A7T  102049_16   Device            40V   

              APPLICATION TECHNOLOGY PROCESSCLASS PROCESSGROUP  \
0         Power Tools; MS         PT   FET_TRENCH      FET12_7   
34283                  MS         PT   FET_TRENCH      FET12_7   
23503     Power Tools; MS         PT   FET_TRENCH      FET12_7   
39633     Power Tools; MS         PT   FET_TRENCH      FET12_7   
11183                  MS         PT   FET_TRENCH      FET12_7   
39061                  MS         PT   FET_TRENCH      FET10_2   
43858     Power Tools; MS         PT   FET_TRENCH      FET12_7   
18678     Power Tools; MS         PT   FET_TRENCH      FET12_7   
25032                  MS         PT   FET_TRENCH      FET10_7   
27314                  MS         PT   FET_TRENCH      FET10_2   
39563                  MS         PT   FET_TRENCH      FET10_7   
8940   Vacuum Cleaner; MS         PT   FET_TRENCH      FET10_7   
6557                   MS         PT   FET_TRENCH      FET10_2   
30521                  MS         PT   FET_TRENCH      F

### Cosine APPROACH

In [6]:
similarity = cosine_similarity(input_data_values, label_encoded_master_data)
nd_list = similarity.tolist()[0]
indices_with_value = heapq.nlargest(20, enumerate(nd_list), key=lambda x: x[1])
indices = [index for index, value in indices_with_value]
values = [value for index, value in indices_with_value]
print(f'Recommended Index {indices}')
print(f'similarity values{values}')
selected_rows = master_data.iloc[indices]
selected_rows

Recommended Index [0, 34283, 23503, 39633, 43858, 11183, 39061, 12901, 18678, 8940, 4979, 27314, 25032, 30872, 30521, 18672, 39563, 43058, 47219, 34144]
similarity values[1.0, 0.9601040423933365, 0.9546565998417026, 0.951252935452332, 0.9452600436198852, 0.9378203063145367, 0.9347500066758165, 0.9341975960767898, 0.932273113359239, 0.927328914142766, 0.9243521198850115, 0.9207833125783301, 0.9194036561285915, 0.9172914327728018, 0.915174927147689, 0.9150732214704155, 0.9147085869881391, 0.9146985378965405, 0.9141571199574502, 0.9113096543815462]


DIVISION_SHORT_DESCR SUB_DIVISION_SHORT_DESCR  BUSINESS_SEGMENT HFGR  \
0                      PSS                       PS                18    A   
34283                  PSS                       PS                18    A   
23503                  PSS                       PS                18    A   
39633                  PSS                       PS                18    A   
43858                  PSS                       PS                15    A   
11183                  PSS                       PS                18    A   
39061                  PSS                       PS                18    A   
12901                  PSS                       PS                15    A   
18678                  PSS                       PS                18    A   
8940                   PSS                       PS                18    A   
4979                   PSS                       PS                18    A   
27314                  PSS                       PS                18    A   
25032                  PSS                       PS                18    A   
30872                  PSS                       PS                18    A   
30521                  PSS                       PS                18    A   
18672                  PSS                       PS                18    A   
39563                  PSS                       PS                18    A   
43058                  PSS                       PS                15    A   
47219                  PSS                       PS                15    A   
34144                  PSS                       PS                18    A   

      HFGR_SHORT_DESCR  FGR   FGR_DESC MPI_TYPE PRODUCT_FAMILY  \
0            Trench LV  A8N  102083_16   Device            75V   
34283        Trench LV  AFV  102627_16   Device            75V   
23503        Trench LV  A8N  102083_16   Device            75V   
39633        Trench LV  A8N  102083_16   Device            75V   
43858       Trench ULV  A87  102067_16   Device            40V   
11183        Trench LV  AD2  102418_16   Device            60V   
39061        Trench LV  A2C  101428_16   Device            55V   
12901       Trench ULV  AMM  104377_16   Device            40V   
18678        Trench LV  A8N  102083_16   Device            75V   
8940         Trench LV  A6M  101996_16   Device            60V   
4979         Trench LV  A8N  102083_16   Device            75V   
27314        Trench LV  A2C  101428_16   Device            55V   
25032        Trench LV  A6Q  102002_16   Device            60V   
30872        Trench LV  A6M  101996_16   Device            60V   
30521        Trench LV  AFV  102627_16   Device            75V   
18672        Trench LV  A6S  102004_16   Device            75V   
39563        Trench LV  A6Q  102002_16   Device            60V   
43058       Trench ULV  AEB  102559_16   Device            40V   
47219       Trench ULV  A87  102067_16   Device            40V   
34144        Trench LV  A6S  102004_16   Device            75V   

              APPLICATION TECHNOLOGY PROCESSCLASS PROCESSGROUP  \
0         Power Tools; MS         PT   FET_TRENCH      FET12_7   
34283                  MS         PT   FET_TRENCH      FET12_7   
23503     Power Tools; MS         PT   FET_TRENCH      FET12_7   
39633     Power Tools; MS         PT   FET_TRENCH      FET12_7   
43858     Power Tools; MS         PT   FET_TRENCH      FET12_7   
11183                  MS         PT   FET_TRENCH      FET12_7   
39061                  MS         PT   FET_TRENCH      FET10_2   
12901     Power Tools; MS         PT   FET_TRENCH      FET12_7   
18678     Power Tools; MS         PT   FET_TRENCH      FET12_7   
8940   Vacuum Cleaner; MS         PT   FET_TRENCH      FET10_7   
4979      Power Tools; MS         PT   FET_TRENCH      FET12_7   
27314                  MS         PT   FET_TRENCH      FET10_2   
25032                  MS         PT   FET_TRENCH      FET10_7   
30872  Vacuum Cleaner; MS         PT   FET_TRENCH      F

# Bert Embedding

### FAISS APPROACH

In [11]:
bert_embedding_master_data = pd.read_csv('bert_embedding/encoded_data.csv')
bertindex = faiss.IndexFlatL2(bert_embedding_master_data.shape[1])
bertindex.add(bert_embedding_master_data)
preprocessed, error = pre_process.bert_conversion(test_input, "inference")
input_data_values = preprocessed.values
start_time = time.time()
distances, indices = bertindex.search(input_data_values, 20)
end_time = time.time()
print(f"Execution Time: {end_time - start_time} seconds")
print(f'Recommended Index {indices.tolist()[0]}')
print(f'Distance List{distances.tolist()[0]}')
selected_rows = master_data.iloc[indices.tolist()[0]]
selected_rows

Execution Time: 0.012230634689331055 seconds
Recommended Index [0, 23503, 18678, 3509, 34283, 24361, 39633, 10042, 8020, 4979, 1609, 35339, 19906, 30521, 30927, 29686, 46540, 20536, 25226, 1941]
Distance List[0.0, 0.37090998888015747, 0.577434778213501, 0.7091246247291565, 0.7556155323982239, 0.8108057975769043, 0.8214480876922607, 0.8803069591522217, 0.9261020421981812, 0.9709787368774414, 1.0278429985046387, 1.0325725078582764, 1.0331907272338867, 1.1480731964111328, 1.154654622077942, 1.1610581874847412, 1.1758936643600464, 1.1771397590637207, 1.2147793769836426, 1.2168354988098145]


DIVISION_SHORT_DESCR SUB_DIVISION_SHORT_DESCR  BUSINESS_SEGMENT HFGR  \
0                      PSS                       PS                18    A   
23503                  PSS                       PS                18    A   
18678                  PSS                       PS                18    A   
3509                   PSS                       PS                18    A   
34283                  PSS                       PS                18    A   
24361                  PSS                       PS                15    A   
39633                  PSS                       PS                18    A   
10042                  PSS                       PS                18    A   
8020                   PSS                       PS                18    A   
4979                   PSS                       PS                18    A   
1609                   PSS                       PS                15    A   
35339                  PSS                       PS                18    A   
19906                  PSS                       PS                15    A   
30521                  PSS                       PS                18    A   
30927                  PSS                       PS                18    A   
29686                  PSS                       PS                18    A   
46540                  PSS                       PS                18    A   
20536                  PSS                       PS                18    A   
25226                  PSS                       PS                15    A   
1941                   PSS                       PS                18    A   

      HFGR_SHORT_DESCR  FGR   FGR_DESC MPI_TYPE PRODUCT_FAMILY  \
0            Trench LV  A8N  102083_16   Device            75V   
23503        Trench LV  A8N  102083_16   Device            75V   
18678        Trench LV  A8N  102083_16   Device            75V   
3509         Trench LV  A8N  102083_16   Device            75V   
34283        Trench LV  AFV  102627_16   Device            75V   
24361       Trench ULV  A87  102067_16   Device            40V   
39633        Trench LV  A8N  102083_16   Device            75V   
10042        Trench LV  A8J  102079_16   Device            60V   
8020         Trench LV  A8N  102083_16   Device            75V   
4979         Trench LV  A8N  102083_16   Device            75V   
1609        Trench ULV  A8D  102073_16   Device            40V   
35339        Trench LV  AFP  102621_16   Device            60V   
19906       Trench ULV  A8B  102071_16   Device            40V   
30521        Trench LV  AFV  102627_16   Device            75V   
30927        Trench LV  A8N  102083_16   Device            75V   
29686        Trench LV  A8N  102083_16   Device            75V   
46540        Trench LV  A8J  102079_16   Device            60V   
20536        Trench LV  A8O  102084_16   Device            75V   
25226       Trench ULV  A87  102067_16   Device            40V   
1941         Trench LV  AFV  102627_16   Device            75V   

           APPLICATION TECHNOLOGY PROCESSCLASS PROCESSGROUP  \
0      Power Tools; MS         PT   FET_TRENCH      FET12_7   
23503  Power Tools; MS         PT   FET_TRENCH      FET12_7   
18678  Power Tools; MS         PT   FET_TRENCH      FET12_7   
3509   Power Tools; MS         PT   FET_TRENCH      FET12_7   
34283               MS         PT   FET_TRENCH      FET12_7   
24361  Power Tools; MS         PT   FET_TRENCH      FET12_7   
39633  Power Tools; MS         PT   FET_TRENCH      FET12_7   
10042  Power Tools; MS         PT   FET_TRENCH      FET12_7   
8020   Power Tools; MS         PT   FET_TRENCH      FET12_7   
4979   Power Tools; MS         PT   FET_TRENCH      FET12_7   
1609   Power Tools; MS         PT   FET_TRENCH      FET12_7   
35339               MS         PT   FET_TRENCH      FET12_7   
19906  Power Tools; MS         PT   FET_TRENCH      FET12_7   
30521               MS         PT   FET_TRENCH      FET12_7   
30927  Power Tools; MS         PT  

### Cosine APPROACH

In [12]:
start_time = time.time()
similarity = cosine_similarity(input_data_values, bert_embedding_master_data)
nd_list = similarity.tolist()[0]
indices_with_value = heapq.nlargest(20, enumerate(nd_list), key=lambda x: x[1])
indices = [index for index, value in indices_with_value]
values = [value for index, value in indices_with_value]
end_time = time.time()
print(f"Execution Time: {end_time - start_time} seconds")
print(f'Recommended Index {indices}')
print(f'similarity values{values}')
selected_rows = master_data.iloc[indices]
selected_rows

Execution Time: 0.3243083953857422 seconds
Recommended Index [0, 23503, 18678, 3509, 34283, 24361, 39633, 10042, 8020, 4979, 1609, 35339, 19906, 30521, 29686, 30927, 46540, 20536, 25226, 1941]
similarity values[1.0000000000000002, 0.9977892866164719, 0.9965220827265696, 0.9957322395253191, 0.9955190201664235, 0.9951176272164872, 0.9950565060396387, 0.9947025644420779, 0.9944642124181718, 0.9941429374032615, 0.9938122083708028, 0.993771352888215, 0.9937619289287498, 0.9932892910044351, 0.9930932493246305, 0.9930265774533582, 0.9929375344369121, 0.9929274823746228, 0.9926692368635931, 0.9926586349203738]


DIVISION_SHORT_DESCR SUB_DIVISION_SHORT_DESCR  BUSINESS_SEGMENT HFGR  \
0                      PSS                       PS                18    A   
23503                  PSS                       PS                18    A   
18678                  PSS                       PS                18    A   
3509                   PSS                       PS                18    A   
34283                  PSS                       PS                18    A   
24361                  PSS                       PS                15    A   
39633                  PSS                       PS                18    A   
10042                  PSS                       PS                18    A   
8020                   PSS                       PS                18    A   
4979                   PSS                       PS                18    A   
1609                   PSS                       PS                15    A   
35339                  PSS                       PS                18    A   
19906                  PSS                       PS                15    A   
30521                  PSS                       PS                18    A   
29686                  PSS                       PS                18    A   
30927                  PSS                       PS                18    A   
46540                  PSS                       PS                18    A   
20536                  PSS                       PS                18    A   
25226                  PSS                       PS                15    A   
1941                   PSS                       PS                18    A   

      HFGR_SHORT_DESCR  FGR   FGR_DESC MPI_TYPE PRODUCT_FAMILY  \
0            Trench LV  A8N  102083_16   Device            75V   
23503        Trench LV  A8N  102083_16   Device            75V   
18678        Trench LV  A8N  102083_16   Device            75V   
3509         Trench LV  A8N  102083_16   Device            75V   
34283        Trench LV  AFV  102627_16   Device            75V   
24361       Trench ULV  A87  102067_16   Device            40V   
39633        Trench LV  A8N  102083_16   Device            75V   
10042        Trench LV  A8J  102079_16   Device            60V   
8020         Trench LV  A8N  102083_16   Device            75V   
4979         Trench LV  A8N  102083_16   Device            75V   
1609        Trench ULV  A8D  102073_16   Device            40V   
35339        Trench LV  AFP  102621_16   Device            60V   
19906       Trench ULV  A8B  102071_16   Device            40V   
30521        Trench LV  AFV  102627_16   Device            75V   
29686        Trench LV  A8N  102083_16   Device            75V   
30927        Trench LV  A8N  102083_16   Device            75V   
46540        Trench LV  A8J  102079_16   Device            60V   
20536        Trench LV  A8O  102084_16   Device            75V   
25226       Trench ULV  A87  102067_16   Device            40V   
1941         Trench LV  AFV  102627_16   Device            75V   

           APPLICATION TECHNOLOGY PROCESSCLASS PROCESSGROUP  \
0      Power Tools; MS         PT   FET_TRENCH      FET12_7   
23503  Power Tools; MS         PT   FET_TRENCH      FET12_7   
18678  Power Tools; MS         PT   FET_TRENCH      FET12_7   
3509   Power Tools; MS         PT   FET_TRENCH      FET12_7   
34283               MS         PT   FET_TRENCH      FET12_7   
24361  Power Tools; MS         PT   FET_TRENCH      FET12_7   
39633  Power Tools; MS         PT   FET_TRENCH      FET12_7   
10042  Power Tools; MS         PT   FET_TRENCH      FET12_7   
8020   Power Tools; MS         PT   FET_TRENCH      FET12_7   
4979   Power Tools; MS         PT   FET_TRENCH      FET12_7   
1609   Power Tools; MS         PT   FET_TRENCH      FET12_7   
35339               MS         PT   FET_TRENCH      FET12_7   
19906  Power Tools; MS         PT   FET_TRENCH      FET12_7   
30521               MS         PT   FET_TRENCH      FET12_7   
29686  Power Tools; MS         PT  

# FAST TEXT

### FAISS APPROACH

In [4]:
fasttext_master_data = pd.read_csv('fasttext/encoded_data.csv')
fast_index = faiss.IndexFlatL2(fasttext_master_data.shape[1])
fast_index.add(fasttext_master_data)
preprocessed, error = pre_process.fasttext_conversion(test_input, "inference")
input_data_values = preprocessed.values
distances, indices = fast_index.search(input_data_values, 20)
print(f'Recommended Index {indices.tolist()[0]}')
print(f'Distance List{distances.tolist()[0]}')
selected_rows = master_data.iloc[indices.tolist()[0]]
selected_rows

Recommended Index [0, 39633, 18678, 4979, 16880, 23580, 36376, 10042, 9659, 23738, 34283, 39922, 14742, 17738, 15922, 5222, 38302, 33397, 3509, 9424]
Distance List[0.7355849742889404, 150.758056640625, 233.8490447998047, 342.2882385253906, 608.0772705078125, 654.8207397460938, 831.89697265625, 980.5036010742188, 993.04345703125, 1044.0233154296875, 1068.9462890625, 1095.300537109375, 1138.1834716796875, 1149.2127685546875, 1203.7525634765625, 1326.650146484375, 1327.587158203125, 1388.30126953125, 1420.6175537109375, 1459.3433837890625]


DIVISION_SHORT_DESCR SUB_DIVISION_SHORT_DESCR  BUSINESS_SEGMENT HFGR  \
0                      PSS                       PS                18    A   
39633                  PSS                       PS                18    A   
18678                  PSS                       PS                18    A   
4979                   PSS                       PS                18    A   
16880                  PSS                       PS                18    A   
23580                  PSS                       PS                18    A   
36376                  PSS                       PS                18    A   
10042                  PSS                       PS                18    A   
9659                   PSS                       PS                18    A   
23738                  PSS                       PS                18    A   
34283                  PSS                       PS                18    A   
39922                  PSS                       PS                18    A   
14742                  PSS                       PS                18    A   
17738                  PSS                       PS                18    A   
15922                  PSS                       PS                18    A   
5222                   PSS                       PS                18    A   
38302                  PSS                       PS                18    A   
33397                  PSS                       PS                18    A   
3509                   PSS                       PS                18    A   
9424                   PSS                       PS                18    A   

      HFGR_SHORT_DESCR  FGR   FGR_DESC MPI_TYPE PRODUCT_FAMILY  \
0            Trench LV  A8N  102083_16   Device            75V   
39633        Trench LV  A8N  102083_16   Device            75V   
18678        Trench LV  A8N  102083_16   Device            75V   
4979         Trench LV  A8N  102083_16   Device            75V   
16880        Trench LV  A8I  102078_16   Device            60V   
23580        Trench LV  A8H  102077_16   Device            60V   
36376        Trench LV  A8K  102080_16   Device            60V   
10042        Trench LV  A8J  102079_16   Device            60V   
9659         Trench LV  A8L  102081_16   Device            60V   
23738        Trench LV  A8J  102079_16   Device            60V   
34283        Trench LV  AFV  102627_16   Device            75V   
39922        Trench LV  A8I  102078_16   Device            60V   
14742        Trench LV  A8H  102077_16   Device            60V   
17738        Trench LV  A6Y  102011_16   Device            75V   
15922        Trench LV  ZE7  102539_16   Device            60V   
5222         Trench LV  AFQ  102622_16   Device            60V   
38302        Trench LV  A8K  102080_16   Device            60V   
33397        Trench LV  A6Y  102011_16   Device            75V   
3509         Trench LV  A8N  102083_16   Device            75V   
9424         Trench LV  A1A  101344_16   Device            75V   

            APPLICATION TECHNOLOGY PROCESSCLASS PROCESSGROUP      SALES_NAME  \
0       Power Tools; MS         PT   FET_TRENCH      FET12_7     IRFB7734PBF   
39633   Power Tools; MS         PT   FET_TRENCH      FET12_7     IRFB7734PBF   
18678   Power Tools; MS         PT   FET_TRENCH      FET12_7     IRFB7734PBF   
4979    Power Tools; MS         PT   FET_TRENCH      FET12_7     IRFB7734PBF   
16880   Power Tools; MS         PT   FET_TRENCH      FET12_7     IRFB7537PBF   
23580  Power Supply; MS         PT   FET_TRENCH      FET12_7     IRFB7534PBF   
36376    Industrial; MS         PT   FET_TRENCH      FET12_7     IRFB7545PBF   
10042   Power Tools; MS         PT   FET_TRENCH      FET12_7     IRFB7540PBF   
9659                 MS         PT   FET_TRENCH      FET12_7     IRFB7546PBF   
23738   Power Tools; MS         PT   FET_TRENCH      FET12_7     IRFB7540PBF   
34283                MS         PT   FET_TRENCH      FET12_7  IRFS7734TRLPBF   
39922   Power Tools;

### Cosine APPROACH

In [5]:
similarity = cosine_similarity(input_data_values, fasttext_master_data)
nd_list = similarity.tolist()[0]
indices_with_value = heapq.nlargest(20, enumerate(nd_list), key=lambda x: x[1])
indices = [index for index, value in indices_with_value]
values = [value for index, value in indices_with_value]
print(f'Recommended Index {indices}')
print(f'similarity values{values}')
selected_rows = master_data.iloc[indices]
selected_rows

Recommended Index [0, 39633, 18678, 4979, 16880, 23580, 36376, 10042, 9659, 23738, 34283, 17738, 39922, 14742, 15922, 5222, 38302, 33397, 9424, 39980]
similarity values[0.999979727799029, 0.995847175585623, 0.9935908534643259, 0.9905205893839949, 0.983571963270891, 0.9821556306620214, 0.9775606818331969, 0.9741799101754555, 0.9737081994292612, 0.9724865810478578, 0.9707412780370791, 0.970160145064021, 0.9697930853431173, 0.9684841310300053, 0.9667411142941028, 0.9636294512547889, 0.9633828788143406, 0.962709057186927, 0.9625947760017053, 0.9607124578221239]


DIVISION_SHORT_DESCR SUB_DIVISION_SHORT_DESCR  BUSINESS_SEGMENT HFGR  \
0                      PSS                       PS                18    A   
39633                  PSS                       PS                18    A   
18678                  PSS                       PS                18    A   
4979                   PSS                       PS                18    A   
16880                  PSS                       PS                18    A   
23580                  PSS                       PS                18    A   
36376                  PSS                       PS                18    A   
10042                  PSS                       PS                18    A   
9659                   PSS                       PS                18    A   
23738                  PSS                       PS                18    A   
34283                  PSS                       PS                18    A   
17738                  PSS                       PS                18    A   
39922                  PSS                       PS                18    A   
14742                  PSS                       PS                18    A   
15922                  PSS                       PS                18    A   
5222                   PSS                       PS                18    A   
38302                  PSS                       PS                18    A   
33397                  PSS                       PS                18    A   
9424                   PSS                       PS                18    A   
39980                  PSS                       PS                18    A   

      HFGR_SHORT_DESCR  FGR   FGR_DESC MPI_TYPE PRODUCT_FAMILY  \
0            Trench LV  A8N  102083_16   Device            75V   
39633        Trench LV  A8N  102083_16   Device            75V   
18678        Trench LV  A8N  102083_16   Device            75V   
4979         Trench LV  A8N  102083_16   Device            75V   
16880        Trench LV  A8I  102078_16   Device            60V   
23580        Trench LV  A8H  102077_16   Device            60V   
36376        Trench LV  A8K  102080_16   Device            60V   
10042        Trench LV  A8J  102079_16   Device            60V   
9659         Trench LV  A8L  102081_16   Device            60V   
23738        Trench LV  A8J  102079_16   Device            60V   
34283        Trench LV  AFV  102627_16   Device            75V   
17738        Trench LV  A6Y  102011_16   Device            75V   
39922        Trench LV  A8I  102078_16   Device            60V   
14742        Trench LV  A8H  102077_16   Device            60V   
15922        Trench LV  ZE7  102539_16   Device            60V   
5222         Trench LV  AFQ  102622_16   Device            60V   
38302        Trench LV  A8K  102080_16   Device            60V   
33397        Trench LV  A6Y  102011_16   Device            75V   
9424         Trench LV  A1A  101344_16   Device            75V   
39980        Trench LV  A1A  101344_16   Device            75V   

            APPLICATION TECHNOLOGY PROCESSCLASS PROCESSGROUP      SALES_NAME  \
0       Power Tools; MS         PT   FET_TRENCH      FET12_7     IRFB7734PBF   
39633   Power Tools; MS         PT   FET_TRENCH      FET12_7     IRFB7734PBF   
18678   Power Tools; MS         PT   FET_TRENCH      FET12_7     IRFB7734PBF   
4979    Power Tools; MS         PT   FET_TRENCH      FET12_7     IRFB7734PBF   
16880   Power Tools; MS         PT   FET_TRENCH      FET12_7     IRFB7537PBF   
23580  Power Supply; MS         PT   FET_TRENCH      FET12_7     IRFB7534PBF   
36376    Industrial; MS         PT   FET_TRENCH      FET12_7     IRFB7545PBF   
10042   Power Tools; MS         PT   FET_TRENCH      FET12_7     IRFB7540PBF   
9659                 MS         PT   FET_TRENCH      FET12_7     IRFB7546PBF   
23738   Power Tools; MS         PT   FET_TRENCH      FET12_7     IRFB7540PBF   
34283                MS         PT   FET_TRENCH      FET12_7  IRFS7734TRLPBF   
17738    Industrial;

# Word 2 VEC

### FAISS APPROACH

In [18]:
word2vec_master_data = pd.read_csv('word2vec/encoded_data.csv')
word2vec_index = faiss.IndexFlatL2(word2vec_master_data.shape[1])
word2vec_index.add(word2vec_master_data)
preprocessed, error = pre_process.word2vec_conversion(test_input, "inference")
input_data_values = preprocessed.values
distances, indices = word2vec_index.search(input_data_values, 20)
print(f'Recommended Index {indices.tolist()[0]}')
print(f'Distance List{distances.tolist()[0]}')
selected_rows = master_data.iloc[indices.tolist()[0]]
selected_rows

/home/emphas_ai/PycharmProjects/LLM_3.12/recommendation_system/similarity_matching/data_preprocess.py:131: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  if logic == "training":


Recommended Index [0, 16880, 25669, 9025, 9992, 39633, 18678, 37817, 27537, 23580, 39142, 23233, 9659, 43173, 22222, 33449, 34283, 46623, 36376, 9424]
Distance List[0.0, 212.9769287109375, 222.73988342285156, 234.3526611328125, 263.92401123046875, 340.68231201171875, 348.7714538574219, 361.13897705078125, 397.95916748046875, 471.0704345703125, 486.85577392578125, 493.4079284667969, 507.814208984375, 509.4071350097656, 570.2064819335938, 584.11279296875, 597.0428466796875, 602.3821411132812, 608.7210693359375, 625.7443237304688]


DIVISION_SHORT_DESCR SUB_DIVISION_SHORT_DESCR  BUSINESS_SEGMENT HFGR  \
0                      PSS                       PS                18    A   
16880                  PSS                       PS                18    A   
25669                  PSS                       PS                18    A   
9025                   PSS                       PS                18    A   
9992                   PSS                       PS                18    A   
39633                  PSS                       PS                18    A   
18678                  PSS                       PS                18    A   
37817                  PSS                       PS                18    A   
27537                  PSS                       PS                18    A   
23580                  PSS                       PS                18    A   
39142                  PSS                       PS                18    A   
23233                  PSS                       PS                18    A   
9659                   PSS                       PS                18    A   
43173                  PSS                       PS                18    A   
22222                  PSS                       PS                18    A   
33449                  PSS                       PS                18    A   
34283                  PSS                       PS                18    A   
46623                  PSS                       PS                18    A   
36376                  PSS                       PS                18    A   
9424                   PSS                       PS                18    A   

      HFGR_SHORT_DESCR  FGR   FGR_DESC MPI_TYPE PRODUCT_FAMILY  \
0            Trench LV  A8N  102083_16   Device            75V   
16880        Trench LV  A8I  102078_16   Device            60V   
25669        Trench LV  A8I  102078_16   Device            60V   
9025         Trench LV  A8I  102078_16   Device            60V   
9992         Trench LV  A8L  102081_16   Device            60V   
39633        Trench LV  A8N  102083_16   Device            75V   
18678        Trench LV  A8N  102083_16   Device            75V   
37817        Trench LV  A8K  102080_16   Device            60V   
27537        Trench LV  A8K  102080_16   Device            60V   
23580        Trench LV  A8H  102077_16   Device            60V   
39142        Trench LV  A8H  102077_16   Device            60V   
23233        Trench LV  A8H  102077_16   Device            60V   
9659         Trench LV  A8L  102081_16   Device            60V   
43173        Trench LV  A8K  102080_16   Device            60V   
22222        Trench LV  A1A  101344_16   Device            75V   
33449        Trench LV  A8I  102078_16   Device            60V   
34283        Trench LV  AFV  102627_16   Device            75V   
46623        Trench LV  A8L  102081_16   Device            60V   
36376        Trench LV  A8K  102080_16   Device            60V   
9424         Trench LV  A1A  101344_16   Device            75V   

            APPLICATION TECHNOLOGY PROCESSCLASS PROCESSGROUP      SALES_NAME  \
0       Power Tools; MS         PT   FET_TRENCH      FET12_7     IRFB7734PBF   
16880   Power Tools; MS         PT   FET_TRENCH      FET12_7     IRFB7537PBF   
25669   Power Tools; MS         PT   FET_TRENCH      FET12_7     IRFB7537PBF   
9025    Power Tools; MS         PT   FET_TRENCH      FET12_7     IRFB7537PBF   
9992                 MS         PT   FET_TRENCH      FET12_7     IRFB7546PBF   
39633   Power Tools; MS         PT   FET_TRENCH      FET12_7     IRFB7734PBF   
18678   Power Tools; MS         PT   FET_TRENCH      FET12_7     IRFB7734PBF   
37817    Industrial; MS         PT   FET_TRENCH      FET12_7     IRFB7545PBF   
27537    Industrial; MS         PT   FET_TRENCH      FET12_7     IRFB7545PBF   
23580  Power Supply; MS         PT   FET_TRENCH      FET12_7     IRFB7534PBF   
39142  Power Supply; MS         PT   FET_TRENCH      FET12_7     IRFB7534PBF   
23233  Power Supply;

### Cosine APPROACH

In [19]:
similarity = cosine_similarity(input_data_values, word2vec_master_data)
nd_list = similarity.tolist()[0]
indices_with_value = heapq.nlargest(20, enumerate(nd_list), key=lambda x: x[1])
indices = [index for index, value in indices_with_value]
values = [value for index, value in indices_with_value]
print(f'Recommended Index {indices}')
print(f'similarity values{values}')
selected_rows = master_data.iloc[indices]
selected_rows

Recommended Index [0, 16880, 25669, 9025, 9992, 39633, 18678, 37817, 27537, 23580, 39142, 23233, 9659, 43173, 22222, 34283, 33449, 39980, 9424, 36376]
similarity values[1.0000000000000002, 0.9912479710829952, 0.9908518843904185, 0.9903597173276356, 0.9891727743586534, 0.9862832329256541, 0.9858316315739079, 0.985151444299803, 0.9836811967280369, 0.9805079985706399, 0.9798521300179133, 0.9795735433423453, 0.9794450085811405, 0.9792342779237408, 0.9780151011919801, 0.976004036972039, 0.9758648177950082, 0.9755552316579761, 0.9754619661701339, 0.9753197338209479]


DIVISION_SHORT_DESCR SUB_DIVISION_SHORT_DESCR  BUSINESS_SEGMENT HFGR  \
0                      PSS                       PS                18    A   
16880                  PSS                       PS                18    A   
25669                  PSS                       PS                18    A   
9025                   PSS                       PS                18    A   
9992                   PSS                       PS                18    A   
39633                  PSS                       PS                18    A   
18678                  PSS                       PS                18    A   
37817                  PSS                       PS                18    A   
27537                  PSS                       PS                18    A   
23580                  PSS                       PS                18    A   
39142                  PSS                       PS                18    A   
23233                  PSS                       PS                18    A   
9659                   PSS                       PS                18    A   
43173                  PSS                       PS                18    A   
22222                  PSS                       PS                18    A   
34283                  PSS                       PS                18    A   
33449                  PSS                       PS                18    A   
39980                  PSS                       PS                18    A   
9424                   PSS                       PS                18    A   
36376                  PSS                       PS                18    A   

      HFGR_SHORT_DESCR  FGR   FGR_DESC MPI_TYPE PRODUCT_FAMILY  \
0            Trench LV  A8N  102083_16   Device            75V   
16880        Trench LV  A8I  102078_16   Device            60V   
25669        Trench LV  A8I  102078_16   Device            60V   
9025         Trench LV  A8I  102078_16   Device            60V   
9992         Trench LV  A8L  102081_16   Device            60V   
39633        Trench LV  A8N  102083_16   Device            75V   
18678        Trench LV  A8N  102083_16   Device            75V   
37817        Trench LV  A8K  102080_16   Device            60V   
27537        Trench LV  A8K  102080_16   Device            60V   
23580        Trench LV  A8H  102077_16   Device            60V   
39142        Trench LV  A8H  102077_16   Device            60V   
23233        Trench LV  A8H  102077_16   Device            60V   
9659         Trench LV  A8L  102081_16   Device            60V   
43173        Trench LV  A8K  102080_16   Device            60V   
22222        Trench LV  A1A  101344_16   Device            75V   
34283        Trench LV  AFV  102627_16   Device            75V   
33449        Trench LV  A8I  102078_16   Device            60V   
39980        Trench LV  A1A  101344_16   Device            75V   
9424         Trench LV  A1A  101344_16   Device            75V   
36376        Trench LV  A8K  102080_16   Device            60V   

            APPLICATION TECHNOLOGY PROCESSCLASS PROCESSGROUP      SALES_NAME  \
0       Power Tools; MS         PT   FET_TRENCH      FET12_7     IRFB7734PBF   
16880   Power Tools; MS         PT   FET_TRENCH      FET12_7     IRFB7537PBF   
25669   Power Tools; MS         PT   FET_TRENCH      FET12_7     IRFB7537PBF   
9025    Power Tools; MS         PT   FET_TRENCH      FET12_7     IRFB7537PBF   
9992                 MS         PT   FET_TRENCH      FET12_7     IRFB7546PBF   
39633   Power Tools; MS         PT   FET_TRENCH      FET12_7     IRFB7734PBF   
18678   Power Tools; MS         PT   FET_TRENCH      FET12_7     IRFB7734PBF   
37817    Industrial; MS         PT   FET_TRENCH      FET12_7     IRFB7545PBF   
27537    Industrial; MS         PT   FET_TRENCH      FET12_7     IRFB7545PBF   
23580  Power Supply; MS         PT   FET_TRENCH      FET12_7     IRFB7534PBF   
39142  Power Supply; MS         PT   FET_TRENCH      FET12_7     IRFB7534PBF   
23233  Power Supply;

In [17]:
import numpy as np
import pandas as pd

# Given lists
label_faiss = [0, 34283, 23503, 39633, 11183, 39061, 43858, 18678, 25032, 27314, 39563, 8940, 6557, 30521, 18672, 31481, 43058, 4979, 34144, 24031]
label_cosine = [0, 34283, 23503, 39633, 43858, 11183, 39061, 12901, 18678, 8940, 4979, 27314, 25032, 30872, 30521, 18672, 39563, 43058, 47219, 34144]
bert_faiss = [0, 23503, 18678, 3509, 34283, 24361, 39633, 10042, 8020, 4979, 1609, 35339, 19906, 30521, 30927, 29686, 46540, 20536, 25226, 1941]
bert_cosine = [0, 23503, 18678, 3509, 34283, 24361, 39633, 10042, 8020, 4979, 1609, 35339, 19906, 30521, 29686, 30927, 46540, 20536, 25226, 1941]
fasttext_faiss = [0, 39633, 18678, 4979, 16880, 23580, 36376, 10042, 9659, 23738, 34283, 39922, 14742, 17738, 15922, 5222, 38302, 33397, 3509, 9424]
fasttext_cosine = [0, 39633, 18678, 4979, 16880, 23580, 36376, 10042, 9659, 23738, 34283, 17738, 39922, 14742, 15922, 5222, 38302, 33397, 9424, 39980]
word2vec_faiss = [0, 16880, 25669, 9025, 9992, 39633, 18678, 37817, 27537, 23580, 39142, 23233, 9659, 43173, 22222, 33449, 34283, 46623, 36376, 9424]
word2vec_cosine = [0, 16880, 25669, 9025, 9992, 39633, 18678, 37817, 27537, 23580, 39142, 23233, 9659, 43173, 22222, 34283, 33449, 39980, 9424, 36376]

# Convert lists to sets
sets = {
    "label_faiss": set(label_faiss),
    "label_cosine": set(label_cosine),
    "bert_faiss": set(bert_faiss),
    "bert_cosine": set(bert_cosine),
    "fasttext_faiss": set(fasttext_faiss),
    "fasttext_cosine": set(fasttext_cosine),
    "word2vec_faiss": set(word2vec_faiss),
    "word2vec_cosine": set(word2vec_cosine)
}

# List of list names to make matrix headers
list_names = list(sets.keys())

# Create an empty matrix to store the size of intersections
matrix = np.zeros((len(list_names), len(list_names)), dtype=int)

# Calculate the intersections and populate the matrix
for i, list_1 in enumerate(list_names):
    for j, list_2 in enumerate(list_names):
        intersection = sets[list_1].intersection(sets[list_2])
        matrix[i, j] = len(intersection)

# Convert matrix to a DataFrame
df = pd.DataFrame(matrix, index=list_names, columns=list_names)

# Display the DataFrame
print("Common Element Matrix (Intersection Sizes):")
df

Common Element Matrix (Intersection Sizes):


label_faiss  label_cosine  bert_faiss  bert_cosine  \
label_faiss               20            17           7            7   
label_cosine              17            20           7            7   
bert_faiss                 7             7          20           20   
bert_cosine                7             7          20           20   
fasttext_faiss             5             5           7            7   
fasttext_cosine            5             5           6            6   
word2vec_faiss             4             4           4            4   
word2vec_cosine            4             4           4            4   

                 fasttext_faiss  fasttext_cosine  word2vec_faiss  \
label_faiss                   5                5               4   
label_cosine                  5                5               4   
bert_faiss                    7                6               4   
bert_cosine                   7                6               4   
fasttext_faiss               20               19               9   
fasttext_cosine              19               20               9   
word2vec_faiss                9                9              20   
word2vec_cosine               9               10              19   

                 word2vec_cosine  
label_faiss                    4  
label_cosine                   4  
bert_faiss                     4  
bert_cosine                    4  
fasttext_faiss                 9  
fasttext_cosine               10  
word2vec_faiss                19  
word2vec_cosine               20